In [6]:
import pandas as pd
import math
from time import sleep
from opencage.geocoder import OpenCageGeocode
from clean import *
import json

In [2]:
df = pd.read_csv('data/profile_data.csv')

In [3]:
df = clean(df)

CLEANING DATA
...
FOUND 56783 ENTRIES
REMOVED 0 EMPTY ENTRIES
REMOVED 0 DUPLICATED ENTRIES


In [4]:
stats(df)

CALCULATING STATS
...
FOUND 56783 ENTRIES
56335 ENTRIES HAVE AGE DATA (99.2%)
19773 ENTRIES HAVE COLLEGE DATA (34.8%)
14456 ENTRIES HAVE CITY DATA (25.5%)
14540 ENTRIES HAVE JOB DATA (25.6%)
10153 ENTRIES HAVE GENDER DATA (17.9%)
55631 ENTRIES HAVE DISTANCE DATA (98.0%)
42256 ENTRIES HAVE DETAIL DATA (74.4%)
15078 ENTRIES HAVE ANTHEM DATA (26.6%)
----
AVERAGE AGE IN DATA SET: 24
NUMBER OF UNIQUE COLLEGES: 7793
NUMBER OF UNIQUE JOBS: 9530
NUMBER OF UNIQUE CITIES: 2793


In [8]:
with open('location_dict.json','r') as file:
    location_dict = json.load(file)

In [21]:
location_dict['Manhattan'][0]['geometry']['lat']
location_dict['Manhattan'][0]['geometry']['lng']

{'lat': 40.7896239, 'lng': -73.9598939}

In [22]:
df.head()

,name,age,college,job,city,gender,distance,details,anthem,profile_pic_urls
0,Gianna,19.0,NaN,NaN,Barnegat,NaN,2430,ig- _.giaannaa_\r\nsc- gianna8378,"('Slide (Remix) (feat. Pop Smoke, A Boogie Wit...",['https://images-ssl.gotinder.com/5dc0814dfc09...
1,Lauren,21.0,Florida State University,NaN,NaN,NaN,1981,A meme,NaN,['https://images-ssl.gotinder.com/5e8a352520cc...
2,Britanny,19.0,Santa Ana College,Boys and girls club,Garden Grove,NaN,19,Get to know me ☺️ Never been to Disneyland 😅 S...,NaN,['https://images-ssl.gotinder.com/5cace3ad9b7a...
3,Kayla,28.0,School,NaN,Los Angeles,Woman,18,"I am a creative person that loves simple, inno...",NaN,['https://images-ssl.gotinder.com/5e90cb67da1c...
4,Kimberly,18.0,"California State University, Los Angeles",NaN,NaN,Woman,15,Just for fun\r\nI’m 5’2,NaN,['https://images-ssl.gotinder.com/5e0ffa53349f...


In [37]:
cities_in_df = list(set(df.city.to_list()))

In [28]:
listed = list(location_dict)

In [42]:
df['in'] = df.city.apply(lambda x: True if (x in listed) else False)

In [48]:
still_need = list(set(df[-df['in']].city.to_list()))

In [49]:
len(still_need)

300

In [50]:
got = list(set(df[df['in']].city.to_list()))

In [51]:
len(got)

2493

In [78]:
loc_data = {}
need_data = []
for each in got:
    try:
        loc_data[each] = location_dict[each][0]['geometry']
    except:
        need_data.append(each)

In [80]:
len(loc_data)

2476

In [72]:
for each in location_dict:
    if len(each) == 1:
        print(each)

In [83]:
with open('loc_data.json','w') as file:
    json.dump(loc_data,file)

In [109]:
def get_location(city):
    try:
        return (loc_data[city]['lat'],loc_data[city]['lng'])
    except:
        return float("nan")

In [110]:
df['location'] = df.city.apply(lambda x: get_location(x))

In [111]:
df.head()

,name,age,college,job,city,gender,distance,details,anthem,profile_pic_urls,in,location
0,Gianna,19.0,NaN,NaN,Barnegat,NaN,2430,ig- _.giaannaa_\r\nsc- gianna8378,"('Slide (Remix) (feat. Pop Smoke, A Boogie Wit...",['https://images-ssl.gotinder.com/5dc0814dfc09...,True,"(39.7557462, -74.2207174)"
1,Lauren,21.0,Florida State University,NaN,NaN,NaN,1981,A meme,NaN,['https://images-ssl.gotinder.com/5e8a352520cc...,False,NaN
2,Britanny,19.0,Santa Ana College,Boys and girls club,Garden Grove,NaN,19,Get to know me ☺️ Never been to Disneyland 😅 S...,NaN,['https://images-ssl.gotinder.com/5cace3ad9b7a...,True,"(33.7746292, -117.9463717)"
3,Kayla,28.0,School,NaN,Los Angeles,Woman,18,"I am a creative person that loves simple, inno...",NaN,['https://images-ssl.gotinder.com/5e90cb67da1c...,True,"(34.0536909, -118.2427666)"
4,Kimberly,18.0,"California State University, Los Angeles",NaN,NaN,Woman,15,Just for fun\r\nI’m 5’2,NaN,['https://images-ssl.gotinder.com/5e0ffa53349f...,False,NaN


In [118]:
import folium
from folium.plugins import HeatMap
import numpy as np

def generateBaseMap(default_location=[37.793331, -122.392776], default_zoom_start=12):
    # Generates Base Folium Map

    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

def graph_it(data):
    # Takes in data Dataframe of lat and lon
    # Returns heatmap of coordinates

#     start = [np.average(data.lat),np.average(data.lon)]
    base_map = generateBaseMap()
    new = data[-data.city.isna()]
    new = new[-new.location.isna()]
    new['lat'] = new.location.apply(lambda x: x[0])
    new['lon'] = new.location.apply(lambda x: x[1])
    new['count'] = 1
    print(new.head())
    HeatMap(data=new[['lat', 'lon', 'count']].groupby(['lat', 'lon']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)
    return base_map
    
    

In [119]:
graph_it(df)

        name   age                college                  job          city  \
0     Gianna  19.0                    NaN                  NaN      Barnegat   
2   Britanny  19.0      Santa Ana College  Boys and girls club  Garden Grove   
3      Kayla  28.0                 School                  NaN   Los Angeles   
9      София  18.0                    NaN                  NaN  Новороссийск   
10      Jena  29.0  Pasadena City College                  NaN   Los Angeles   

   gender distance                                            details  \
0     NaN     2430                 ig- _.giaannaa_\r\nsc- gianna8378    
2     NaN       19  Get to know me ☺️ Never been to Disneyland 😅 S...   
3   Woman       18  I am a creative person that loves simple, inno...   
9     NaN     6772  Ласка одной львицы лучше, чем гарем дешевых гиен    
10  Woman       15  Oh, how much I do not like the type of men who...   

                                               anthem  \
0   ('Slide (Remix) (fe